In [37]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Dropout

In [38]:
flights_df=pd.read_csv('./Resources/Atlanta_Flight_Status.csv').drop('Unnamed: 0',axis=1)

flights_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,ORIGIN_CITY_NAME,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DELAY_GROUP,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,1,7,AA,1006,"Miami, FL","Atlanta, GA",2100,2058.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
1,1,1,7,AA,1045,"Miami, FL","Atlanta, GA",1925,1922.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
2,1,1,7,AA,1051,"Atlanta, GA","Dallas/Fort Worth, TX",1817,1816.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
3,1,1,7,AA,1051,"Dallas/Fort Worth, TX","Atlanta, GA",1425,1420.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
4,1,1,7,AA,1151,"Atlanta, GA","Miami, FL",602,553.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593798,9,30,6,WN,778,"Indianapolis, IN","Atlanta, GA",810,806.0,0.0,-1.0,2,NaN,NaN,NaN,NaN,NaN
593799,9,30,6,WN,854,"Miami, FL","Atlanta, GA",510,505.0,0.0,-1.0,3,NaN,NaN,NaN,NaN,NaN
593800,9,30,6,WN,888,"Baltimore, MD","Savannah, GA",1505,1527.0,22.0,1.0,3,NaN,NaN,NaN,NaN,NaN
593801,9,30,6,WN,924,"Atlanta, GA","West Palm Beach/Palm Beach, FL",1055,1058.0,3.0,0.0,3,NaN,NaN,NaN,NaN,NaN


In [39]:
flights_df.describe()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DELAY_GROUP,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,593803.000000,593803.000000,593803.000000,593803.000000,593803.000000,587287.000000,587287.000000,587287.000000,593803.000000,121231.000000,121231.00000,121231.000000,121231.000000,121231.000000
mean,4.893544,15.837980,3.979845,2110.437495,1345.712629,1344.986775,15.866375,0.213134,3.181274,30.026231,4.26852,12.346479,0.113040,22.934654
std,2.504191,8.772622,1.993397,1342.466807,511.045747,527.795295,52.036827,2.322178,1.892323,79.210780,32.03912,30.364191,3.125093,51.516719
min,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,-2.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,3.000000,8.000000,2.000000,1054.000000,910.000000,910.000000,0.000000,-1.000000,2.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,5.000000,16.000000,4.000000,2034.000000,1327.000000,1329.000000,0.000000,-1.000000,3.000000,8.000000,0.00000,0.000000,0.000000,0.000000
75%,7.000000,23.000000,6.000000,2727.000000,1755.000000,1759.000000,10.000000,0.000000,4.000000,29.000000,0.00000,15.000000,0.000000,26.000000
max,9.000000,31.000000,7.000000,9685.000000,2359.000000,2400.000000,3221.000000,12.000000,11.000000,3221.000000,1197.00000,1300.000000,323.000000,2216.000000


In [40]:
flights_df['MKT_UNIQUE_CARRIER'].unique()

array(['AA', 'AS', 'B6', 'DL', 'F9', 'NK', 'UA', 'WN', 'G4'], dtype=object)

In [41]:
#Only look at flights leaving Atlanta on Delta, AA, WN
DL_flights_df = flights_df.loc[(flights_df['ORIGIN_CITY_NAME']=='Atlanta, GA')]
#Seperate the data into features and targets.
X=DL_flights_df[['MONTH','DAY_OF_MONTH','DAY_OF_WEEK','DEST_CITY_NAME','CRS_DEP_TIME','MKT_UNIQUE_CARRIER']]

X=X.reset_index(drop=True)
X

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEST_CITY_NAME,CRS_DEP_TIME,MKT_UNIQUE_CARRIER
0,1,1,7,"Dallas/Fort Worth, TX",1817,AA
1,1,1,7,"Miami, FL",602,AA
2,1,1,7,"Washington, DC",1424,AA
3,1,1,7,"Miami, FL",1822,AA
4,1,1,7,"Charlotte, NC",1253,AA
...,...,...,...,...,...,...
280337,9,30,6,"Milwaukee, WI",825,WN
280338,9,30,6,"Phoenix, AZ",1755,WN
280339,9,30,6,"Orlando, FL",1030,WN
280340,9,30,6,"West Palm Beach/Palm Beach, FL",1055,WN


In [42]:
dates = []
for row in X.iterrows():
    if len(str(row[1]['DAY_OF_MONTH'])) == 1:
        dates.append(f"0{int(row[1]['MONTH'])}-0{int(row[1]['DAY_OF_MONTH'])}")
    else:
        dates.append(f"0{int(row[1]['MONTH'])}-{int(row[1]['DAY_OF_MONTH'])}")
        
X['Date'] = dates

In [43]:
X=X.drop(['MONTH','DAY_OF_MONTH'],axis = 1)
X

,DAY_OF_WEEK,DEST_CITY_NAME,CRS_DEP_TIME,MKT_UNIQUE_CARRIER,Date
0,7,"Dallas/Fort Worth, TX",1817,AA,01-01
1,7,"Miami, FL",602,AA,01-01
2,7,"Washington, DC",1424,AA,01-01
3,7,"Miami, FL",1822,AA,01-01
4,7,"Charlotte, NC",1253,AA,01-01
...,...,...,...,...,...
280337,6,"Milwaukee, WI",825,WN,09-30
280338,6,"Phoenix, AZ",1755,WN,09-30
280339,6,"Orlando, FL",1030,WN,09-30
280340,6,"West Palm Beach/Palm Beach, FL",1055,WN,09-30


In [44]:
X["DAY_OF_WEEK"].unique()

array([7, 1, 2, 3, 4, 5, 6], dtype=int64)

In [45]:
day_of_weeks = []
days_of_the_week = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
for row in X.iterrows():
    day_of_weeks.append(days_of_the_week[row[1]['DAY_OF_WEEK']-1])

In [46]:
X['DAY_OF_WEEK'] = day_of_weeks
X

,DAY_OF_WEEK,DEST_CITY_NAME,CRS_DEP_TIME,MKT_UNIQUE_CARRIER,Date
0,Sunday,"Dallas/Fort Worth, TX",1817,AA,01-01
1,Sunday,"Miami, FL",602,AA,01-01
2,Sunday,"Washington, DC",1424,AA,01-01
3,Sunday,"Miami, FL",1822,AA,01-01
4,Sunday,"Charlotte, NC",1253,AA,01-01
...,...,...,...,...,...
280337,Saturday,"Milwaukee, WI",825,WN,09-30
280338,Saturday,"Phoenix, AZ",1755,WN,09-30
280339,Saturday,"Orlando, FL",1030,WN,09-30
280340,Saturday,"West Palm Beach/Palm Beach, FL",1055,WN,09-30


In [47]:
X = pd.get_dummies(X)

In [48]:
X

,CRS_DEP_TIME,DAY_OF_WEEK_Friday,DAY_OF_WEEK_Monday,DAY_OF_WEEK_Saturday,DAY_OF_WEEK_Sunday,DAY_OF_WEEK_Thursday,DAY_OF_WEEK_Tuesday,DAY_OF_WEEK_Wednesday,"DEST_CITY_NAME_Albany, GA","DEST_CITY_NAME_Albany, NY",...,Date_09-21,Date_09-22,Date_09-23,Date_09-24,Date_09-25,Date_09-26,Date_09-27,Date_09-28,Date_09-29,Date_09-30
0,1817,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,602,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1424,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1822,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1253,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280337,825,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
280338,1755,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
280339,1030,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
280340,1055,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [49]:
X.to_csv('./predictions/Resources/X.csv')

In [50]:
#Scale the feature data, X
from sklearn.preprocessing import StandardScaler
# Scaling the numeric columns
scaler = StandardScaler()
scaler.fit(X)
transformed_data = scaler.transform(X)
# Creating a DataFrame with with the scaled data
transformed_data_df = pd.DataFrame(transformed_data)

transformed_data_df

,0,1,2,3,4,5,6,7,8,9,...,427,428,429,430,431,432,433,434,435,436
0,0.714998,-0.422606,-0.416191,-0.380501,2.443438,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,-0.055277
1,-1.693681,-0.422606,-0.416191,-0.380501,2.443438,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,-0.055277
2,-0.064106,-0.422606,-0.416191,-0.380501,2.443438,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,-0.055277
3,0.724910,-0.422606,-0.416191,-0.380501,2.443438,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,-0.055277
4,-0.403105,-0.422606,-0.416191,-0.380501,2.443438,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,-0.055277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280337,-1.251594,-0.422606,-0.416191,2.628114,-0.409259,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,18.090587
280338,0.592086,-0.422606,-0.416191,2.628114,-0.409259,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,18.090587
280339,-0.845192,-0.422606,-0.416191,2.628114,-0.409259,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,18.090587
280340,-0.795630,-0.422606,-0.416191,2.628114,-0.409259,-0.422501,-0.399989,-0.405981,-0.050708,-0.054886,...,-0.060578,-0.0604,-0.05531,-0.060191,-0.060518,-0.057505,-0.057661,-0.060637,-0.06037,18.090587


In [51]:
#Seperate the data into features and targets
is_delayed=DL_flights_df['DEP_DELAY_GROUP'].tolist()
y=[]
#Determine if the flight was delayed using the DEP_DELAY_NEW column and
#convert to boolean values
for i in is_delayed:
    if i < 0:
        y.append(0)
    else:
        y.append(1)
y = pd.DataFrame(y)
y.value_counts()

0    141774
1    138568
dtype: int64

In [52]:
y.to_csv('./predictions/Resources/y.csv')

In [53]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(transformed_data_df, y, random_state=78)

In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="tanh", input_dim=437))
nn_model.add(Dropout(0.2))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))
nn_model.add(Dropout(0.2))
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="sigmoid"))
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="sigmoid"))
nn_model.add(Dropout(0.2))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                28032     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 64)               

In [55]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

Epoch 1/10
6571/6571 [==============================] - 7s 956us/step - loss: 0.6294 - accuracy: 0.6451
Epoch 2/10
6571/6571 [==============================] - 6s 941us/step - loss: 0.6128 - accuracy: 0.6626
Epoch 3/10
6571/6571 [==============================] - 6s 943us/step - loss: 0.6062 - accuracy: 0.6691
Epoch 4/10
6571/6571 [==============================] - 6s 927us/step - loss: 0.6015 - accuracy: 0.6745
Epoch 5/10
6571/6571 [==============================] - 6s 915us/step - loss: 0.5970 - accuracy: 0.6777
Epoch 6/10
6571/6571 [==============================] - 6s 914us/step - loss: 0.5942 - accuracy: 0.6799
Epoch 7/10
6571/6571 [==============================] - 6s 913us/step - loss: 0.5910 - accuracy: 0.6834
Epoch 8/10
6571/6571 [==============================] - 6s 910us/step - loss: 0.5877 - accuracy: 0.6853
Epoch 9/10
6571/6571 [==============================] - 6s 907us/step - loss: 0.5856 - accuracy: 0.6885
Epoch 10/10
6571/6571 [==============================] - 6s 909u

In [57]:
# Check the model's loss and accuracy 
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2191/2191 - 1s - loss: 0.6026 - accuracy: 0.6700 - 1s/epoch - 514us/step
Loss: 0.6026403903961182, Accuracy: 0.6700196862220764


In [58]:
nn_model.save('./predictions/Resources/nn_model.keras')

In [59]:
y_test.value_counts()

0    35486
1    34600
dtype: int64

In [60]:
#Compare the test data to the predictions
predictions = nn_model.predict(X_test).reshape(70086).tolist()
predictions_df = pd.DataFrame({'y_test':y_test[0],'predictions':predictions})

2191/2191 [==============================] - 1s 535us/step


In [61]:
predictions_df.describe()

,y_test,predictions
count,70086.000000,70086.000000
mean,0.493679,0.483974
std,0.499964,0.215646
min,0.000000,0.159892
25%,0.000000,0.300108
50%,0.000000,0.479806
75%,1.000000,0.636982
max,1.000000,0.954172


In [62]:
predictions_df.sort_values(by = 'predictions').tail(50)

,y_test,predictions
74442,1,0.953570
102820,1,0.953570
73660,1,0.953576
228214,1,0.953576
227376,1,0.953588
102380,1,0.953589
102152,1,0.953594
212597,1,0.953597
102148,1,0.953601
233414,1,0.953604


In [63]:
delayed_predictions = []
for i in predictions:
    if (i >=.5):
        delayed_predictions.append(i)
delayed_predictions

[0.9037230014801025,
 0.5235322713851929,
 0.7050780653953552,
 0.5051313638687134,
 0.6279612183570862,
 0.523289680480957,
 0.9049326181411743,
 0.7419788837432861,
 0.7021300792694092,
 0.8676320314407349,
 0.9025132656097412,
 0.9498350024223328,
 0.5436857342720032,
 0.7356787919998169,
 0.5830093622207642,
 0.5545126795768738,
 0.7888759970664978,
 0.6721972823143005,
 0.5334741473197937,
 0.9521692991256714,
 0.8593934774398804,
 0.8074341416358948,
 0.5424816012382507,
 0.509739875793457,
 0.663175642490387,
 0.9291249513626099,
 0.7756483554840088,
 0.6741408109664917,
 0.52327960729599,
 0.6993818283081055,
 0.5257307291030884,
 0.7371193766593933,
 0.8750987648963928,
 0.6918084621429443,
 0.623211145401001,
 0.5717104077339172,
 0.5630156397819519,
 0.5399131178855896,
 0.6169647574424744,
 0.7787429094314575,
 0.5010929107666016,
 0.9532938003540039,
 0.7533174753189087,
 0.6452177166938782,
 0.8818790316581726,
 0.8675462603569031,
 0.6308680176734924,
 0.6759994626045227

In [64]:
len(delayed_predictions)

31801

## LogisticRegression

In [65]:

from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression

In [66]:
model = LogisticRegression(random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

C:\Users\Matt\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

In [67]:
model.score(X_train, y_train)

0.660575679172057

In [68]:
# Make a prediction using the testing data
predictions = model.predict(X_test)
predictions

array([0, 1, 1, ..., 0, 1, 1], dtype=int64)

In [69]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test,predictions)

0.6605722789317875

In [70]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,predictions)

array([[23967, 11519],
       [12257, 22343]], dtype=int64)

In [71]:
# Print the classification report for the model
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
accuracy = (tp + tn) / (tp + fp + tn + fn)
accuracy

0.6607596381588334